In [1]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment, clean_exp_with_hyphaes
from hyphae_id import relabel_nodes_after_amb, resolve_ambiguity

In [2]:
plate = 13
directory = (
    "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
)
listdir = os.listdir(directory)
list_dir_interest = [
    name
    for name in listdir
    if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
]
ss = [name.split("_")[0] for name in list_dir_interest]
ff = [name.split("_")[1] for name in list_dir_interest]
dates_datetime = [
    datetime(
        year=int(ss[i][:4]),
        month=int(ss[i][4:6]),
        day=int(ss[i][6:8]),
        hour=int(ff[i][0:2]),
        minute=int(ff[i][2:4]),
    )
    for i in range(len(list_dir_interest))
]
dates_datetime.sort()
dates_datetime_chosen = dates_datetime[1:25]
dates = [
    f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}'
    for date in dates_datetime_chosen
]

In [3]:
exp = Experiment(13)
exp.load(dates, local=False, raw=False)

In [4]:
exp_clean = clean_exp_with_hyphaes(exp)

before getting hyphaes True
0.0
0.0658544616397761
0.1317089232795522
0.1975633849193283
0.2634178465591044
0.3292723081988805
0.3951267698386566
0.4609812314784327
0.5268356931182088
0.5926901547579848
0.658544616397761
0.724399078037537
0.7902535396773132
0.8561080013170892
0.9219624629568653
0.9878169245966414
after getting hyphaes True
removing  1824  nodes


In [ ]:
from experiment_class import Node

equ_class, ambig, connection = resolve_ambiguity(exp_clean.hyphaes)
new_graph, newposs = relabel_nodes_after_amb(
    connection, exp_clean.nx_graph, exp_clean.positions
)
exp_relabeled = Experiment(13)
exp_relabeled.copy(exp_clean)
exp_relabeled.nx_graph = new_graph
exp_relabeled.positions = newposs
exp_relabeled.nodes = []
labels = {int(node) for g in exp_relabeled.nx_graph for node in g}
for label in labels:
    exp_relabeled.nodes.append(Node(label, exp_relabeled))
exp_clean_relabeled = clean_exp_with_hyphaes(exp_relabeled)

In [ ]:
get_mother(exp_clean_relabeled.hyphaes)

In [ ]:
hyphs, gr_inf = save_hyphaes(exp_clean_relabeled)
exp_clean_relabeled.save()

In [ ]:
exp_clean_relabeled = clean_exp_with_hyphaes(exp_clean_relabeled)

In [6]:
disapeared = [
    node for node in exp_relabeled.nodes if node not in exp_clean_relabeled.nodes
]

In [7]:
len(disapeared)

79

In [12]:
node = choice(disapeared)
print(node.ts())
plt.close("all")

[17]


In [19]:
def get_mother(hyphaes):
    nodes_within = {hyphae.end: {} for hyphae in hyphaes}
    for i, hyphae in enumerate(hyphaes):
        if i % 500 == 0:
            print(i / len(hyphaes))
        mothers = []
        t0 = hyphae.ts[0]
        for hyph in hyphaes:
            if t0 in hyph.ts and hyph.end != hyphae.end:
                if t0 in nodes_within[hyph.end].keys():
                    nodes_within_hyph = nodes_within[hyph.end][t0]
                else:
                    nodes_within_hyph = hyph.get_nodes_within(t0)[0]
                    nodes_within[hyph.end][t0] = nodes_within_hyph
                if hyphae.root.label in nodes_within_hyph:
                    mothers.append(hyph)
        hyphae.mother = mothers

In [387]:
def get_pixel_growth_and_new_children(hyphae, t1, t2):
    assert t1 < t2, "t1 should be strictly inferior to t2"
    edges = hyphae.get_nodes_within(t2)[1]
    mini = np.inf
    if t1 not in hyphae.ts:
        pixels = []
        nodes = [hyphae.root]
        for edge in edges:
            pixels.append(edge.pixel_list(t2))
            nodes.append(edge.end)
        return (pixels, nodes)
    else:
        if len(edges) == 0:
            print(hyphae.root, hyphae.end)
            return ([], [])
        for i, edge in enumerate(edges):
            distance = np.min(
                np.linalg.norm(
                    hyphae.end.pos(t1) - np.array(edge.pixel_list(t2)), axis=1
                )
            )
            if distance < mini:
                index = i
                mini = distance
                last_edge = edge
                index_nearest_pixel = np.argmin(
                    np.linalg.norm(
                        hyphae.end.pos(t1) - np.array(edge.pixel_list(t2)), axis=1
                    )
                )
        pixels = [last_edge.pixel_list(t2)[index_nearest_pixel:]]
        nodes = [-1, last_edge.end]
        for edge in edges[index + 1 :]:
            pixels.append(edge.pixel_list(t2))
            nodes.append(edge.end)
        return (pixels, nodes)

In [393]:
def save_hyphaes(exp, path="Data/"):
    column_names_hyphaes = ["end", "root", "ts", "mother"]
    column_names_growth_info = [
        "hyphae",
        "t",
        "t+1",
        "nodes_in_hyphae",
        "segment_of_growth_t_t+1",
        "node_list_t_t+1",
    ]
    hyphaes = pd.DataFrame(columns=column_names_hyphaes)
    growth_info = pd.DataFrame(columns=column_names_growth_info)
    for hyph in exp.hyphaes:
        new_line_hyphae = pd.DataFrame(
            {
                "end": [hyph.end.label],
                "root": [hyph.root.label],
                "ts": [hyph.ts],
                "mother": [-1 if len(hyph.mother) == 0 else hyph.mother[0].end.label],
            }
        )  # index 0 for
        # mothers need to be modified to resolve multi mother issue
        hyphaes = hyphaes.append(new_line_hyphae, ignore_index=True)
        for index in range(len(hyph.ts[:-1])):
            t = hyph.ts[index]
            tp1 = hyph.ts[index + 1]
            pixels, nodes = get_pixel_growth_and_new_children(hyph, t, tp1)
            new_line_growth = pd.DataFrame(
                {
                    "hyphae": [hyph.end],
                    "t": [t],
                    "t+1": [tp1],
                    "nodes_in_hyphae": [hyph.get_nodes_within(t)],
                    "segment_of_growth_t_t+1": [pixels],
                    "node_list_t_t+1": [nodes],
                }
            )
            growth_info.append(new_line_growth, ignore_index=True)
    path = "Data/"
    hyphs.to_csv(path + f"hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv")
    gr_inf.to_csv(path + f"growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv")
    sio.savemat(
        path + f"hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat",
        {name: col.values for name, col in hyphs.items()},
    )
    sio.savemat(
        path + f"growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat",
        {name: col.values for name, col in gr_inf.items()},
    )
    return (hyphaes, growth_info)

In [ ]:
# def prune_graph(nx_graph,threshold=150):
#     S = [nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph)]
#     len_connected=[len(nx_graph.nodes) for nx_graph in S]
#     return(len_connected)
# for i,nx_g in enumerate(exp_clean_relabeled.nx_graph):
#     print(prune_graph(nx_g))

In [392]:
path = "Data/"
hyphs.to_csv(path + f"hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv")
gr_inf.to_csv(path + f"growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv")
sio.savemat(
    path + f"hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat",
    {name: col.values for name, col in hyphs.items()},
)
sio.savemat(
    path + f"growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat",
    {name: col.values for name, col in gr_inf.items()},
)

In [343]:
exp_clean_relabeled.plot([6, 7], [[hyph.end.label, hyph.root.label]] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
has_mother = [hypha for hypha in exp_clean_relabeled.hyphaes if len(hypha.mother) >= 1]

In [30]:
len(exp_clean_relabeled.hyphaes)

2655

In [66]:
doesn_have_mother = [
    hypha for hypha in exp_clean_relabeled.hyphaes if len(hypha.mother) == 0
]

In [168]:
hyphae = choice(doesn_have_mother)
plt.close("all")
hyphae, hyphae.ts

(Hyphae(134,367),
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23])

In [179]:
t=8
hypo=[hyph for hyph in exp_clean_relabeled.hyphaes if t in hyph.ts and 367 in hyph.get_nodes_within(t)[0]][]

[Hyphae(21,5394), Hyphae(134,367)]

In [184]:
for t in range(23):
    if exp_clean_relabeled.get_node(5394).is_in(t):
        print(t, exp_clean_relabeled.get_node(5394).degree(t))

8 3
9 3
10 3
11 3
12 3
13 3
14 3
15 3
16 3
17 3
18 3
19 3
20 3
21 3
22 3


In [187]:
for t in range(23):
    if exp_clean_relabeled.get_node(417).is_in(t):
        print(t, exp_clean_relabeled.get_node(417).degree(t))

0 4
1 3
2 4
3 3
4 3
5 3
6 4
7 4
8 3
9 3
10 3
11 3
12 3
13 3
14 3
15 3
16 3
17 3
18 3
19 3
20 3
21 3
22 3


In [196]:
len(exp_clean_relabeled.get_node(5394).edges(8)[2].pixel_list(8))

21

In [198]:
exp_clean_relabeled.get_node(5394).edges(8)[0].begin.degree(8)

3

In [209]:
def clean_degree_4(nx_graph, pos, thresh=30):
    nx_graph_clean = nx.Graph.copy(nx_graph)
    remaining_to_fuse = True
    while remaining_to_fuse:
        remaining_to_fuse = False
        to_fuse = []
        for edge in nx_graph_clean.edges:
            if (
                nx_graph_clean.get_edge_data(*edge)["weight"] <= thresh
                and nx_graph_clean.degree(edge[0]) == 3
                and nx_graph_clean.degree(edge[1]) == 3
            ):
                to_fuse.append(edge)
        nodes_to_fuse = [edge[0] for edge in to_fuse] + [edge[1] for edge in to_fuse]
        occurence_count = Counter(nodes_to_fuse)
        difficult_cases = []
    #         for edge in to_fuse:
    #             node1=edge[0]
    #             node2=edge[1]
    #             if occurence_count[node1]!=1 and occurence_count[node2]!=1:
    #                 difficult_cases.append(edge)
    #             else:
    #                 remaining_to_fuse=True
    #                 if occurence_count[node1]==1:
    #                     pivot = node2
    #                     fuser = node1
    #                 else:
    #                     pivot = node1
    #                     fuser = node2
    #                 neighbours = list(nx_graph_clean.neighbors(fuser))
    #                 for neighbour in neighbours:
    #                     right_n = pivot
    #                     left_n = neighbour
    #                     right_edge = nx_graph_clean.get_edge_data(fuser,right_n)['pixel_list']
    #                     left_edge = nx_graph_clean.get_edge_data(fuser,left_n)['pixel_list']
    #                     if np.any(right_edge[0]!=pos[fuser]):
    #                         right_edge = list(reversed(right_edge))
    #                     if np.any(left_edge[-1]!=pos[fuser]):
    #                         left_edge = list(reversed(left_edge))
    #                     pixel_list = left_edge+right_edge[1:]
    #                     info={'weight':len(pixel_list),'pixel_list':pixel_list}
    #                     if right_n!=left_n:
    #                         connection_data=nx_graph_clean.get_edge_data(right_n,left_n)
    #                         if connection_data is None or connection_data['weight']>=info['weight']:
    #                             if not connection_data is None:
    #                                 nx_graph_clean.remove_edge(right_n,left_n)
    #                             nx_graph_clean.add_edges_from([(right_n,left_n,info)])
    #                 nx_graph_clean.remove_node(fuser)
    #         print(len(difficult_cases))
    #     print(len(nx_graph_clean.nodes))
    #     return(nx_graph_clean,difficult_cases)
    return occurence_count

In [210]:
to_f = clean_degree_4(exp.nx_graph[8], exp.positions[8])

In [213]:
len(to_f.keys())

215

In [189]:
np.linalg.norm(
    exp_clean_relabeled.get_node(5394).pos(10)
    - exp_clean_relabeled.get_node(417).pos(10)
)

28.600699292150182

In [186]:
exp.plot(
    [6, 7, 8], [[hyphae.end.label, hyphae.root.label, 21, 5394, 367, 359, 417]] * 3
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
exp_clean_relabeled.plot([11, 12, 13], [[hyphae.end.label, hyphae.root.label]] * 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
hyphae = choice(has_mother)
plt.close("all")
hyphae, hyphae.ts, hyphae.mother, hyphae.mother[0].ts

(Hyphae(3336,3335),
 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
 [Hyphae(37,48)],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23])

In [65]:
exp_clean_relabeled.plot(
    [2, 3, 4],
    [
        [
            hyphae.end.label,
            hyphae.root.label,
            hyphae.mother[0].end.label,
            hyphae.mother[0].root.label,
        ]
    ]
    * 3,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
exp_clean.plot([17, 18], [[node.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
exp_clean_relabeled.plot([17, 18], [[node.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def final_clean(exp):
    cleaning = True
    while cleaning:
        exp_clean = clean_exp_with_hyphaes(exp)
        equ_class, ambig, connection = resolve_ambiguity(exp_clean.hyphaes)
        new_graph, newposs = relabel_nodes_after_amb(
            connection, exp_clean.nx_graph, exp_clean.positions
        )
        exp_relabeled = Experiment(13)
        exp_relabeled.copy(exp_clean)
        exp_relabeled.nx_graph = new_graph
        exp_relabeled.positions = newposs
        exp_relabeled.nodes = []
        labels = {int(node) for g in exp_relabeled.nx_graph for node in g}
        for label in labels:
            exp_relabeled.nodes.append(Node(label, exp_relabeled))
        exp = exp_relabeled

In [5]:
for hyphae in exp_clean.hyphaes:
    hyphae.update_ts()

In [11]:
def get_disapearing_roots(hyphaes):
    problems = []
    for hyph in hyphaes:
        for t in hyph.ts:
            if t not in hyph.root.ts() or t not in hyph.end.ts():
                problems.append(hyph)
                continue
    return problems

In [146]:
len(connection.keys())

3006

In [65]:
len([label for label in connection.keys() if connection[label] != label])

429

In [ ]:
new_graph, newposs = relabel_nodes_after_amb(
    connection, exp_clean.nx_graph, exp_clean.positions
)

In [ ]:
labels = {int(node) for g in exp_relabeled.nx_graph for node in g}

In [ ]:
exp_relabeled = Experiment(13)
exp_relabeled.copy(exp_clean)
exp_relabeled.nx_graph = new_graph
exp_relabeled.positions = newposs
exp_relabeled.nodes = []
labels = {int(node) for g in exp_relabeled.nx_graph for node in g}
for label in labels:
    exp_relabeled.nodes.append(Node(label, exp_relabeled))
exp_clean_relabeled = clean_exp_with_hyphaes(exp_relabeled)

before getting hyphaes True
0.0
0.06422607578676943
0.12845215157353887
0.1926782273603083
0.25690430314707774
0.32113037893384716
0.3853564547206166
0.449582530507386
0.5138086062941555
0.5780346820809249
0.6422607578676943
0.7064868336544637
0.7707129094412332
0.8349389852280026
0.899165061014772
0.9633911368015414
after getting hyphaes True
removing  158  nodes
removing False
before pruning True
after pruning True


In [186]:
len(exp_clean_relabeled.nodes), len(exp_clean.nodes), len(exp.nodes)

(8536, 9044, 11036)

In [187]:
disapeared = [node for node in exp_clean_relabeled.nodes if node not in exp_clean.nodes]

In [191]:
labels = [node.label for node in exp_clean_relabeled.nodes]
labels2 = [node.label for node in exp_clean.nodes]

In [192]:
np.mean(labels), np.mean(labels2)

(13051.53725398313, 5449.03339230429)

In [188]:
len(disapeared)

5959

In [156]:
exp_relabeled_clean_relabeled.get_node(38060).ts()

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [162]:
exp_relabeled_clean_relabeled.plot([20, 21], [[38060, 42610, 38015, 38017, 42600]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
equ_class, ambig, connection = resolve_ambiguity(exp_clean_relabeled.hyphaes)
new_graph, newposs = relabel_nodes_after_amb(
    connection, exp_clean_relabeled.nx_graph, exp_clean_relabeled.positions
)
exp_relabeled_clean_relabeled = Experiment(13)
exp_relabeled_clean_relabeled.copy(exp_clean_relabeled)
exp_relabeled_clean_relabeled.nx_graph = new_graph
exp_relabeled_clean_relabeled.positions = newposs
exp_relabeled_clean_relabeled.nodes = []
labels = {int(node) for g in exp_relabeled_clean_relabeled.nx_graph for node in g}
for label in labels:
    exp_relabeled_clean_relabeled.nodes.append(
        Node(label, exp_relabeled_clean_relabeled)
    )
exp_clean_relabeled_clean_relabeled = clean_exp_with_hyphaes(
    exp_relabeled_clean_relabeled
)

before getting hyphaes False
0.0
0.07507507507507508
0.15015015015015015
0.22522522522522523
0.3003003003003003
0.37537537537537535
20 38060 42610
20 38060 38015
20 38060 38017
20 38060 42600
20 38060 42610
20 38060 38015
20 38060 38017
20 38060 42600
20 38060 42610
20 38060 38015
20 38060 38017
0.45045045045045046
0.5255255255255256
0.6006006006006006
0.6756756756756757
0.7507507507507507
20 9496 38017
20 9496 42600
20 9496 42610
20 9496 38015
20 9496 38017
20 9496 42600
20 9496 42610
20 9496 38015
20 9496 38017
20 9496 42600
20 9496 42610
0.8258258258258259
0.9009009009009009
0.975975975975976
after getting hyphaes False
removing  110  nodes
removing False
before pruning False
after pruning False
before changing nx_graph False
after changing nx_graph False


In [ ]:
exp_clean_relabeled_clean_relabeled.hyphaes

In [172]:
equ_class, ambig, connection = resolve_ambiguity(
    exp_clean_relabeled_clean_relabeled.hyphaes
)
new_graph, newposs = relabel_nodes_after_amb(
    connection,
    exp_clean_relabeled_clean_relabeled.nx_graph,
    exp_clean_relabeled_clean_relabeled.positions,
)
exp_relabeled_clean_relabeled.nx_graph = new_graph
exp_relabeled_clean_relabeled.positions = newposs
exp_relabeled_clean_relabeled.nodes = []
labels = {int(node) for g in exp_relabeled_clean_relabeled.nx_graph for node in g}
for label in labels:
    exp_relabeled_clean_relabeled.nodes.append(
        Node(label, exp_relabeled_clean_relabeled)
    )
exp_clean_relabeled_clean_relabeled = clean_exp_with_hyphaes(
    exp_relabeled_clean_relabeled
)

before getting hyphaes False
0.0
0.07507507507507508
0.15015015015015015
0.22522522522522523
0.3003003003003003
0.37537537537537535
0.45045045045045046
0.5255255255255256
0.6006006006006006
0.6756756756756757
0.7507507507507507
0.8258258258258259
20 9496 82161
20 9496 86744
20 9496 86754
20 9496 82159
20 9496 82161
20 9496 86744
20 9496 86754
20 9496 82159
20 9496 82161
20 9496 86744
20 9496 86754
0.9009009009009009
0.975975975975976
20 82204 86754
20 82204 82159
20 82204 82161
20 82204 86744
20 82204 86754
20 82204 82159
20 82204 82161
20 82204 86744
20 82204 86754
20 82204 82159
20 82204 82161
after getting hyphaes False
removing  106  nodes
removing False
before pruning False
after pruning False
before changing nx_graph False
after changing nx_graph False


In [174]:
disapearing = [
    node
    for node in exp_clean_relabeled.nodes
    if node not in exp_clean_relabeled_clean_relabeled.nodes
]

In [176]:
choice(disapearing)

Node(19031)

In [83]:
relabeled = [label for label in connection.keys() if connection[label] != label]

In [93]:
label = connection[choice(relabeled)]
exp_relabeled.get_node(label).ts(), exp_clean.get_node(label).ts()
plt.close("all")

In [96]:
exp_relabeled.plot([14, 15, 16], [[label]] * 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
solved = {x for liste in connection.values() for x in liste}.union(
    {key for key in connection.keys() if len(connection[key]) >= 1}
)

In [15]:
len(solved)

695

In [16]:
len(ambig)

1178

In [310]:
hard_ambig = [
    ambi for ambi in ambig if len(set(ambi[0].ts).intersection(set(ambi[1].ts))) >= 1
]

In [ ]:
set(ambig[0][1].ts)

In [22]:
weird = [
    ambi
    for ambi in ambig
    if len(set(ambi[0].ts).intersection(set(ambi[1].ts))) == 0
    if ambi[0] not in solved
]

In [470]:
inter = [c for c in hard_ambig if len(c[0].ts) >= 2 and len(c[1].ts) >= 2]

In [23]:
len(weird)

19

In [97]:
hyphaes_prob = choice(weird)
plt.close("all")

In [98]:
hyphaes_prob[0].ts, hyphaes_prob[1].ts

([1, 2, 3, 4, 5, 6, 7, 8], [17, 18, 19, 20, 21, 22, 23])

In [102]:
hyphaes_prob[0], hyphaes_prob[1]

(Hyphae(2933,2948), Hyphae(8452,2948))

In [101]:
boundaries = [
    node.label for mother in hyphaes_prob for node in [mother.end, mother.root]
]

In [53]:
np.linalg.norm(hyphaes_prob[0].end.pos(19) - hyphaes_prob[1].end.pos(20))

259.1100152444903

In [536]:
added_labels = [node.label for node in hyphaes_prob[0].end.neighbours(9)] + [
    node.label for node in hyphaes_prob[0].end.neighbours(9)[1].neighbours(10)
]

In [557]:
added_labels = [node.label for node in hyphaes_prob[0].end.neighbours(6)]

In [52]:
exp_clean.get_node(9080).pos(19) // 5

array([ 974, 4607], dtype=int32)

In [50]:
boundaries

[9080, 9079, 9379, 9079]

In [561]:
plt.close("all")

In [69]:
hyphaes_prob[1].end.ts()

[10]

In [106]:
exp_clean.get_node(8452) in solved

True

In [100]:
exp_clean.plot([8, 17], [boundaries] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
exp.plot([8, 17], [boundaries] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
node = hyphaes_prob[0].end
node.show_source_image(8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [462]:
node = hyphaes_prob[1].end
node.show_source_image(22)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
len(exp_clean.hyphaes), len(ambig)

(3448, 0)

In [108]:
hyphae = choice(exp_clean.hyphaes)
plt.close("all")

In [214]:
def get_mother(hyphaes):
    nodes_within = {hyphae.end: {} for hyphae in hyphaes}
    for i, hyphae in enumerate(hyphaes):
        if i % 500 == 0:
            print(i / len(hyphaes))
        mothers = []
        t0 = hyphae.ts[0]
        for hyph in hyphaes:
            if t0 in hyph.ts and hyph.end != hyphae.end:
                if t0 in nodes_within[hyph.end].keys():
                    nodes_within_hyph = nodes_within[hyph.end][t0]
                else:
                    nodes_within_hyph = hyph.get_nodes_within(t0)[0]
                    nodes_within[hyph.end][t0] = nodes_within_hyph
                if hyphae.end.label == 5464:
                    if hyph.end.label == 1622:
                        print(nodes_within_hyph)
                if hyphae.root.label in nodes_within_hyph:
                    mothers.append(hyph)
        hyphae.mother = mothers


#     candidate_mother=[]
#     for hyphae in hyph.experiment.hyphaes:
#         if hyphae.end != hyph.end:
#             found = False
#             for t in hyphae.ts:
#                 if hyphae.root.is_in(t):
#                     if hyph.root.label in hyphae.get_nodes_within(t)[0] and hyph.root.label != hyphae.root.label and not found and hyphae.end.pos(t)[0]<19500:
#                         candidate_mother.append(hyphae)
#                         found = True
#     return(candidate_mother)

In [116]:
# def get_mother(hyph):
#     candidate_mother=[]
#     for hyphae in hyph.experiment.hyphaes:
#         if hyphae.end != hyph.end:
#             found = False
#             for t in hyphae.ts:
#                 if hyphae.root.is_in(t):
#                     if hyph.root.label in hyphae.get_nodes_within(t)[0] and hyph.root.label != hyphae.root.label and not found and hyphae.end.pos(t)[0]<19500:
#                         candidate_mother.append(hyphae)
#                         found = True
#     return(candidate_mother)

In [168]:
# for i in range(20):
#     hyphae=choice(exp_clean.hyphaes)
#     if len(hyphae.ts)>=2:
#         mothers = get_mother(hyphae)
#         print(mothers)
#         if len(mothers)>=2:
#             break

TypeError: 'Hyphae' object is not iterable

In [215]:
get_mother(exp_clean.hyphaes)

0.0
0.14501160092807425
0.2900232018561485
0.43503480278422274
[2322, 2147, 2074, 4843, 4842, 4840, 4838, 5465, 5466, 1622]
0.580046403712297
0.7250580046403712
0.8700696055684455


In [216]:
ambiguous_mothers = [hyphae for hyphae in exp_clean.hyphaes if len(hyphae.mother) >= 2]

630

In [217]:
no_moter = [hyphae for hyphae in exp_clean.hyphaes if len(hyphae.mother) == 0]

679

In [218]:
len(exp_clean.hyphaes)

3448

In [228]:
hyphae = choice(exp_clean.hyphaes)

In [222]:
hyphae

Hyphae(7054,7063)

In [229]:
mothers = hyphae.mother
mothers

[Hyphae(127,491), Hyphae(235,225)]

In [230]:
nodes = [
    node for mother in mothers for node in mother.get_nodes_within(mother.ts[0])[0]
] + hyphae.get_nodes_within(hyphae.ts[0])[0]

In [234]:
hyphae

Hyphae(3099,225)

In [231]:
hyphae.ts

[2, 3, 4, 5, 6, 7]

In [232]:
mothers[0].ts, mothers[1].ts

([0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3])

In [195]:
hyphae_moth = [hyph for hyph in exp_clean.hyphaes if hyph.end.label == 1622][0]

In [199]:
hyphae_moth.get_nodes_within(8)

([2322, 2147, 2074, 4843, 4842, 4840, 4838, 5465, 5466, 1622],
 [Edge(2322,2147),
  Edge(2147,2074),
  Edge(2074,4843),
  Edge(4843,4842),
  Edge(4842,4840),
  Edge(4840,4838),
  Edge(4838,5465),
  Edge(5465,5466),
  Edge(5466,1622)])

In [235]:
exp_clean.plot([7, 8], [[] + nodes] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [237]:
plt.close("all")

In [238]:
exp_clean.plot([3, 4], [boundaries] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [236]:
boundaries = [
    node.label for mother in mothers for node in [mother.end, mother.root]
] + [hyphae.end.label, hyphae.root.label]

In [158]:
hyphae, mothers

(Hyphae(10019,8934),
 [Hyphae(6893,7102),
  Hyphae(8934,7656),
  Hyphae(10324,7102),
  Hyphae(10867,7102)])

In [159]:
mothers[0].ts

[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [ ]:
exp_clean.get_node(1354).ts()

In [ ]:
exp.get_node(1354).ts()

In [81]:
hyphae.root.ts(), hyphae.end.ts()

([6, 7], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
hyphae.sorted(set(hyphae.end.ts()).intersection(set(hyphae.root.ts())))

In [44]:
hyphae

Hyphae(7714,7716)

In [153]:
hyphae.root.ts()

[15, 17, 18]

In [41]:
exp_clean.plot([15, 16, 17], [[hyphae.root.label, hyphae.end.label]] * 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
exp.plot([15, 16, 17], [[hyphae.root.label, hyphae.end.label]] * 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
def clean_exp_with_hyphaes(experiment):
    ts = {}
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
    exp_clean = Experiment(experiment.plate)
    exp_clean.copy(experiment)
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes = []
    for label in labels:
        exp_clean.nodes.append(Node(label, exp_clean))
    for node in exp_clean.nodes:
        ts[node.label] = []
        for t in range(len(nx_graph_cleans)):
            if node.is_in(t):
                ts[node.label].append(t)
    hyphaes, problems = get_hyphae(exp_clean)
    to_remove = []
    safe_nodes = set()
    for hyphae in hyphaes:
        for t in hyphae.end.ts():
            if t in hyphae.root.ts():
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if (
            len(ts[node.label]) == 1
            and ts[node.label][0] != len(experiment.nx_graph) - 1
            and node.label not in safe_nodes
        ):
            to_remove.append(node.label)
    return to_remove

In [49]:
to_rem = clean_exp_with_hyphaes(exp)

before getting hyphaes True
0.0
0.05735589331803843
0.11471178663607685
0.1720676799541153
0.2294235732721537
0.28677946659019216
0.3441353599082306
0.401491253226269
0.4588471465443074
0.5162030398623458
0.5735589331803843
0.6309148264984227
0.6882707198164612
0.7456266131344995
0.802982506452538
0.8603383997705765
0.9176942930886148
0.9750501864066533
after getting hyphaes True
removing  1627  nodes
removing False


TypeError: append() takes exactly one argument (2 given)

In [129]:
for node in to_rem:
    if len(exp_clean.get_node(node).ts()) >= 2:
        print(node)

In [96]:
c = (1, 2)
c.__hash__()

3713081631934410656

In [18]:
node_obj = Node(hyphae.root.label, exp_clean)
node_obj.show_source_image(hyphae.root.ts()[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
i = np.argmax([np.mean(im) for im in ims])

In [80]:
plot_t_tp1([0], [], {0: (posimg[1][i], posimg[0][i])}, None, ims[i], ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
hyphae.get_nodes_within(21)

([13592, 12239], [Edge(13592,12239)])

In [34]:
hyphae.end.neighbours(20)

[Node(12473)]

In [57]:
hyphae

Hyphae(6610,6825)

In [50]:
np.linalg.norm(
    exp.get_node(12239).neighbours(21)[0].pos(21)
    - exp.get_node(12239).neighbours(20)[0].pos(20)
)

53.53503525729669

In [7]:
# hyphae = choice(exp_clean.hyphaes)
# hyphae.get_mother()
# hyphae.mother

AttributeError: 'list' object has no attribute 'label'

In [8]:
def get_mother(hyph):
    candidate_mother = []
    problem = []
    for hyphae in hyph.experiment.hyphaes:
        if hyphae.end != hyph.end:
            found = False
            for t in hyphae.ts():
                if hyphae.root.is_in(t):
                    if (
                        hyph.root.label in hyphae.get_nodes_within(t)[0]
                        and hyph.root.label != hyphae.root.label
                        and found == False
                        and hyphae.end.pos(t)[0] < 19500
                    ):
                        candidate_mother.append(hyphae)
                        found = True
                else:
                    problem.append((hyphae, t))
    return (candidate_mother, problem)

In [ ]:
hyphae.experiment.hyphaes

In [108]:
sorted(set([1, 2, 3]).intersection(set([2, 3, 4])))

[2, 3]

In [16]:
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy, copy
from collections import Counter


class Experiment:
    def __init__(self, plate):
        self.plate = plate
        self.path_param = "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat"

    def load(self, dates, local=False, pickle=False, raw=False):
        self.dates = dates
        self.raw = raw
        if local:
            paths = [
                f"Data/graph_{date}_{self.plate}_full_labeled.csv" for date in dates
            ]
            paths_connection = [
                f"Data/connection_{date}_{self.plate}.csv" for date in dates[:-1]
            ]
        else:
            paths = [
                get_path(date, self.plate, True, extension="_full_labeled.csv")
                for date in dates
            ]
            paths_connection = [
                get_path(date, self.plate, True, extension="_connection.csv")
                for date in dates[:-1]
            ]
        if raw:
            nx_graph_poss = []
            for date in dates:
                nx_graph_poss.append(
                    generate_nx_graph(
                        pd.read_csv(
                            get_path(
                                date,
                                self.plate,
                                True,
                                extension="_raw_aligned_skeleton.csv",
                            ),
                            converters={
                                "origin_pos": transform_list,
                                "end_pos": transform_list,
                                "pixel_list": ast.literal_eval,
                            },
                        ),
                        labeled=True,
                    )
                )
        else:
            nx_graph_poss = []
            for path in paths:
                nx_graph_poss.append(
                    generate_nx_graph(
                        pd.read_csv(
                            path,
                            converters={
                                "origin_pos": transform_list,
                                "end_pos": transform_list,
                                "pixel_list": ast.literal_eval,
                            },
                        ),
                        labeled=True,
                    )
                )
        from_tip_growth_pattern = []
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs = [nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean = []
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected = [len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        skeletons = []
        for nx_graph in nx_graph_clean:
            skeletons.append(generate_skeleton(nx_graph, dim=(20800, 46000)))
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions = poss
        self.nx_graph = nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
        self.skeletons = skeletons
        self.hyphaes = None
        labels = {node for g in self.nx_graph for node in g}
        self.nodes = []
        for label in labels:
            self.nodes.append(Node(label, self))

    def copy(self, experiment):
        self.positions = experiment.positions
        self.nx_graph = experiment.nx_graph
        self.connections = experiment.connections
        self.growth_patterns = experiment.growth_patterns
        self.skeletons = experiment.skeletons
        self.dates = experiment.dates
        self.plate = experiment.plate
        self.nodes = None
        self.hyphaes = None

    def save(self, path=f"Data/"):
        from_tip_growth_pattern_tab = []
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(
                pd.read_csv(
                    get_path(date, self.plate, True, extension="_connection.csv")
                )
            )
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(
                path + f"connection_{date}_{self.plate}.csv"
            )
        tabs_labeled = []
        for date in self.dates:
            tabs_labeled.append(
                pd.read_csv(
                    get_path(date, self.plate, True, extension="_full_labeled.csv"),
                    converters={
                        "origin_pos": transform_list,
                        "end_pos": transform_list,
                        "pixel_list": ast.literal_eval,
                    },
                )
            )
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path + f"graph_{date}_{self.plate}_full_labeled.csv")
            sio.savemat(
                path + f"graph_{date}_{self.plate}_full_labeled.mat",
                {name: col.values for name, col in tabs_labeled[i].items()},
            )

    def pickle_save(self):
        pickle.dump(self, open(f"Data/" + f"experiment_{self.plate}.pick", "wb"))

    def pickle_load(self):
        self = pickle.load(open(f"Data/" + f"experiment_{self.plate}.pick", "rb"))

    def get_node(self, label):
        return Node(label, self)

    def get_edge(self, begin, end):
        return Edge(begin, end, self)

    def compress_skeleton(self, t, factor):
        shape = self.skeletons[t].shape
        final_picture = np.zeros(shape=(shape[0] // factor, shape[1] // factor))
        for pixel in self.skeletons[t].keys():
            x = min(round(pixel[0] / factor), shape[0] // factor - 1)
            y = min(round(pixel[1] / factor), shape[1] // factor - 1)
            final_picture[x, y] += 1
        return final_picture >= 1

    def load_images(self, t):
        date = self.dates[t]
        date_plate = f"/2020{date}"
        plate_str = f"_Plate{self.plate}"
        path_tile = (
            "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
            + date_plate
            + plate_str
            + "/Img/TileConfiguration.registered.txt"
        )
        tileconfig = pd.read_table(
            path_tile,
            sep=";",
            skiprows=4,
            header=None,
            converters={2: ast.literal_eval},
            skipinitialspace=True,
        )
        for name in tileconfig[0]:
            im = imageio.imread(
                "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
                + date_plate
                + plate_str
                + "/Img/"
                + name
            )
            np.save("Temp/" + name, im)

    def get_growing_tips(self, t, threshold=80):
        growths = {
            tip: sum([len(branch) for branch in self.growth_patterns[t][tip]])
            for tip in self.growth_patterns[t].keys()
        }
        growing_tips = [node for node in growths.keys() if growths[node] >= threshold]
        return growing_tips

    def pinpoint_anastomosis(self, t):
        nx_graph_tm1 = self.nx_graph[t]
        nx_grapht = self.nx_graph[t + 1]
        from_tip = self.connections[t]
        pos_tm1 = self.positions[t]
        anastomosis = []
        origins = []
        tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node) == 1]
        number_anastomosis = 0

        def dist_branch(node, nx_graph, pos):
            mini = np.inf
            for edge in nx_graph.edges:
                pixel_list = nx_graph.get_edge_data(*edge)["pixel_list"]
                if (
                    np.linalg.norm(np.array(pixel_list[0]) - np.array(pos[node]))
                    <= 5000
                ):
                    distance = np.min(
                        np.linalg.norm(
                            np.array(pixel_list) - np.array(pos[node]), axis=1
                        )
                    )
                    if distance < mini:
                        mini = distance
            return mini

        def count_neighbors_is_from_root(equ_list, nx_graph, root):
            count = 0
            for neighbor in nx_graph.neighbors(root):
                if neighbor in equ_list:
                    count += 1
            return count

        for tip in tips:
            #         print(tip)
            consequence = from_tip[tip]
            for node in consequence:
                if (
                    node in nx_grapht.nodes
                    and nx_grapht.degree(node) >= 3
                    and count_neighbors_is_from_root(consequence, nx_grapht, node) < 2
                ):
                    #                 if node==2753:
                    #                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
                    #                     print(list(nx_grapht.neighbors(node)))
                    anastomosis.append(node)
                    origins.append(tip)
                    number_anastomosis += 1
            if (
                tip not in nx_grapht.nodes
                and dist_branch(tip, nx_grapht, pos_tm1) <= 30
                and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])[
                    "weight"
                ]
                >= 20
            ):
                origins.append(tip)
                number_anastomosis += 1 / 2
        return (anastomosis, origins, number_anastomosis)

    def find_image_pos(self, ys, xs, t, local=False):
        params = read_mat(self.path_param)
        x1 = params["x1"]
        x2 = params["x2"]
        y0 = params["y0"]
        Sf = params["Sf"]
        L0center = params["L0center"]
        date = self.dates[t]
        date_plate = f"/2020{date}"
        plate_str = f"_Plate{self.plate}"
        findingbaits = read_mat(
            "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
            + date_plate
            + plate_str
            + "/Analysis/FindingBaits.mat"
        )
        Rcenter = findingbaits["Rcenter"]
        Rradii = findingbaits["Rradii"]
        Lcenter = findingbaits["Lcenter"]
        t = findingbaits["t"]
        Lradii = findingbaits["Lradii"]
        path_tile = (
            "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
            + date_plate
            + plate_str
            + "/Img/TileConfiguration.registered.txt"
        )
        tileconfig = pd.read_table(
            path_tile,
            sep=";",
            skiprows=4,
            header=None,
            converters={2: ast.literal_eval},
            skipinitialspace=True,
        )
        xs_yss = list(tileconfig[2])
        xes = [xs_ys[0] for xs_ys in xs_yss]
        yes = [xs_ys[1] for xs_ys in xs_yss]
        cmin = np.min(xes)
        cmax = np.max(xes)
        rmin = np.min(yes)
        rmax = np.max(yes)
        S0 = [rmax + 3000 + abs(rmin), cmax + 4096 + abs(cmin)]
        theta = np.arctan((Rcenter[1] - Lcenter[1]) / (Rcenter[0] - Lcenter[0]))
        Sry = S0[0] * np.cos(abs(theta)) + S0[1] * np.sin(abs(theta))
        C = np.tan(
            -np.arctan(
                (
                    ys
                    + y0
                    - 1
                    - L0center[1]
                    - np.sqrt(Lcenter[0] ** 2 + (S0[0] - Lcenter[1]) ** 2)
                    * np.sin(np.arctan((S0[0] - Lcenter[1]) / Lcenter[0]) + theta)
                )
                / (
                    xs
                    + (x2 + x1) / 2
                    - (Sf[0] - y0)
                    - 1
                    - L0center[0]
                    + np.sqrt(Lcenter[0] ** 2 + (S0[0] - Lcenter[1]) ** 2)
                    * np.cos(np.arctan((S0[0] - Lcenter[1]) / Lcenter[0]) + theta)
                )
            )
            - theta
        )
        ximg = -(
            ys
            + y0
            - 1
            - L0center[1]
            - np.sqrt(Lcenter[0] ** 2 + (S0[0] - Lcenter[1]) ** 2)
            * np.sin(np.arctan((S0[0] - Lcenter[1]) / Lcenter[0]) + theta)
        ) / (np.sqrt(1 + C**2) * np.sin(np.arctan(C) + theta))
        yimg = S0[0] - C * ximg

        def find(xsub, ysub, x, y):
            indexes = []
            for i in range(len(xsub)):
                if (
                    x >= xsub[i] - cmin
                    and x < xsub[i] - cmin + 4096
                    and y >= ysub[i] - rmin
                    and y < ysub[i] - rmin + 3000
                ):
                    indexes.append(i)
            return indexes

        indsImg = find(xes, yes, ximg, yimg)
        possImg = [
            ximg - np.array(xes)[indsImg] + cmin + 1,
            yimg - np.array(yes)[indsImg] + rmin + 1,
        ]
        paths = []
        for index in indsImg:
            if local:
                paths.append("Temp/" + tileconfig[0][index] + ".npy")
            else:
                paths.append(
                    "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
                    + date_plate
                    + plate_str
                    + "/Img/"
                    + tileconfig[0][index]
                )
        if local:
            ims = [np.load(path) for path in paths]
        else:
            ims = [imageio.imread(path) for path in paths]
        return (ims, possImg)

    def plot(self, ts, node_lists=[], shift=(0, 0), compress=5, save="", time=None):
        global check
        right = 0.90
        top = 0.90
        if len(save) >= 1:
            fig = plt.figure(figsize=(14, 12))
            size = 10
        else:
            fig = plt.figure()
            size = 5
        ax = fig.add_subplot(111)
        scale = 1 / len(ts)
        power = len(ts)
        compressed_images = []
        for i, t in enumerate(ts):
            compressed_images.append(self.compress_skeleton(t, compress))
        visibility = [True for t in ts]
        final_image = scale * compressed_images[0] * visibility[0]
        for i, compressed_image in enumerate(compressed_images[1:]):
            final_image = np.power(
                final_image**power
                + ((i + 2) * scale * compressed_image * visibility[i + 1]) ** power,
                1 / power,
            )
        l1 = ax.imshow(final_image, cmap="plasma", interpolation="none", vmin=0, vmax=1)
        rax = plt.axes([0.05, 0.4, 0.1, 0.15])
        labels = [f"{4*t}h" for t in ts]
        check = CheckButtons(rax, labels, visibility)

        def func(label):
            index = labels.index(label)
            visibility[index] = not visibility[index]
            final_image = visibility[0] * scale * compressed_images[0]
            for i, compressed_image in enumerate(compressed_images[1:]):
                final_image = np.power(
                    final_image**power
                    + visibility[i + 1] * ((i + 2) * scale * compressed_image) ** power,
                    1 / power,
                )
            l1.set_data(final_image)
            plt.draw()

        check.on_clicked(func)
        if len(node_lists) > 0:
            for i, node_list in enumerate(node_lists):
                grey = (i + 1) / len(labels)
                bbox = dict(boxstyle="circle", fc=colors.rgb2hex((grey, grey, grey)))
                #             ax.text(right, top, time,
                #                 horizontalalignment='right',
                #                 verticalalignment='bottom',
                #                 transform=ax.transAxes,color='white')
                for node in node_list:
                    #                     print(self.positions[ts[i]])
                    if node in self.positions[ts[i]].keys():
                        t = ax.text(
                            (self.positions[ts[i]][node][1] - shift[1]) // compress,
                            (self.positions[ts[i]][node][0] - shift[0]) // compress,
                            str(node),
                            ha="center",
                            va="center",
                            size=size,
                            bbox=bbox,
                        )
        if len(save) >= 1:
            plt.savefig(save)
            plt.close(fig)
        else:
            plt.show()


class Node:
    def __init__(self, label, experiment):
        self.experiment = experiment
        self.label = label

    def __eq__(self, other):
        return self.label == other.label

    def __repr__(self):
        return f"Node({self.label})"

    def __str__(self):
        return str(self.label)

    def __hash__(self):
        return self.label

    def neighbours(self, t):
        return [
            self.experiment.get_node(node)
            for node in self.experiment.nx_graph[t].neighbors(self.label)
        ]

    def is_in(self, t):
        return self.label in self.experiment.nx_graph[t].nodes

    def degree(self, t):
        return self.experiment.nx_graph[t].degree(self.label)

    def edges(self, t):
        return [
            self.experiment.get_edge(self, neighbour)
            for neighbour in self.neighbours(t)
        ]

    def pos(self, t):
        return self.experiment.positions[t][self.label]

    def ts(self):
        return [t for t in range(len(self.experiment.nx_graph)) if self.is_in(t)]

    def show_source_image(self, t):
        pos = self.pos(t)
        x, y = pos[0], pos[1]
        ims, posimg = self.experiment.find_image_pos(x, y, t)
        i = np.argmax([np.mean(im) for im in ims])
        plot_t_tp1([0], [], {0: (posimg[1][i], posimg[0][i])}, None, ims[i], ims[i])


class Edge:
    def __init__(self, begin, end, experiment):
        self.begin = begin
        self.end = end
        self.experiment = experiment

    def __repr__(self):
        return f"Edge({self.begin},{self.end})"

    def __str__(self):
        return str((self.begin, self.end))

    def is_in(self, t):
        return (self.begin.label, self.end.label) in self.experiment.nx_graph[t].edges

    def pixel_list(self, t):
        return orient(
            self.experiment.nx_graph[t].get_edge_data(self.begin.label, self.end.label)[
                "pixel_list"
            ],
            self.begin.pos(t),
        )

    def orientation_whole(self, t):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1] - pixel_list[0]
        unit_vector = vector / np.linalg.norm(vector)
        vertical_vector = np.array([-1, 0])
        dot_product = np.dot(vertical_vector, unit_vector)
        if (
            vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
        ):  # determinant
            angle = np.arccos(dot_product) / (2 * np.pi) * 360
        else:
            angle = -np.arccos(dot_product) / (2 * np.pi) * 360
        return angle

    def orientation_begin(self, t, length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[min(length, len(pixel_list) - 1)] - pixel_list[0]
        unit_vector = vector / np.linalg.norm(vector)
        vertical_vector = np.array([-1, 0])
        dot_product = np.dot(vertical_vector, unit_vector)
        if (
            vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
        ):  # determinant
            angle = np.arccos(dot_product) / (2 * np.pi) * 360
        else:
            angle = -np.arccos(dot_product) / (2 * np.pi) * 360
        return angle

    def orientation_end(self, t, length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1] - pixel_list[max(0, len(pixel_list) - 1 - length)]
        unit_vector = vector / np.linalg.norm(vector)
        vertical_vector = np.array([-1, 0])
        dot_product = np.dot(vertical_vector, unit_vector)
        if (
            vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
        ):  # determinant
            angle = np.arccos(dot_product) / (2 * np.pi) * 360
        else:
            angle = -np.arccos(dot_product) / (2 * np.pi) * 360
        return angle


class Hyphae:
    def __init__(self, tip):
        self.experiment = tip.experiment
        self.ts = tip.ts()
        self.end = tip
        self.root = None
        self.mother = None

    def __eq__(self, other):
        return self.end.label == other.end.label

    def __repr__(self):
        return f"Hyphae({self.end},{self.root})"

    def __str__(self):
        return str((self.end, self.root))

    def __hash__(self):
        return self.end.label

    def get_edges(self, t, length=20):
        first_neighbour = self.end.neighbours(t)[0]
        last_node = self.end
        current_node = first_neighbour
        current_edge = Edge(last_node, current_node, self.experiment)
        moving_on_hyphae = True
        edges = [current_edge]
        nodes = [last_node, current_node]
        i = 0
        while moving_on_hyphae:
            i += 1
            if i >= 100:
                print(t, self.end, current_node)
            if i >= 110:
                break
            #                 print ('moving',current_node)
            if current_node.degree(t) < 2:
                #                     print(current_node.degree(t),current_node)
                moving_on_hyphae = False
            else:
                maxi = -np.inf
                orientation = current_edge.orientation_end(t, length)
                for neighbours_t in current_node.neighbours(t):
                    #                     print (neighbours_t)
                    candidate_edge = Edge(current_node, neighbours_t, self.experiment)
                    orientation_candidate = candidate_edge.orientation_begin(t, length)
                    angle = np.cos(
                        (orientation - orientation_candidate) / 360 * 2 * np.pi
                    )
                    if angle > maxi:
                        maxi = angle
                        next_node_candidate = neighbours_t
                #                     print(maxi,next_node_candidate)
                candidate_edge = Edge(
                    current_node, next_node_candidate, self.experiment
                )
                orientation_candidate = candidate_edge.orientation_begin(t, length)
                maxi_compet = -np.inf
                #                     print('compet')
                for neighbours_t in current_node.neighbours(t):
                    if neighbours_t != last_node:
                        competitor_edge = Edge(
                            neighbours_t, current_node, self.experiment
                        )
                        orientation_competitor = competitor_edge.orientation_end(
                            t, length
                        )
                        angle = np.cos(
                            (orientation_competitor - orientation_candidate)
                            / 360
                            * 2
                            * np.pi
                        )
                        if angle > maxi_compet:
                            maxi_compet = angle
                            competitor = neighbours_t
                #                             print(neighbours_t,angle)
                #                     print(maxi_compet,competitor)
                if maxi_compet > maxi:
                    moving_on_hyphae = False
                else:
                    last_node, current_node = current_node, next_node_candidate
                    current_edge = Edge(last_node, current_node, self.experiment)
                    edges.append(current_edge)
                    nodes.append(current_node)
        #         while moving:
        #             c= move_hyphae(llast_node,ccurrent_node)
        #             edges += c[0]
        #             nodes += c[1]
        #             competitor = c[2]
        # #             print('moving back', nodes[-1],competitor)
        #             move_backward = move_hyphae(nodes[-1],competitor)
        #             end_node_move_backward = move_backward[1][-1]
        #             if end_node_move_backward in nodes:
        # #                 print('restarting',competitor,nodes[-1])
        #                 llast_node,ccurrent_node = c[2],nodes[-1]
        #             else:
        #                 moving=False
        root = nodes[-1]
        edges = edges
        nodes = nodes
        return (root, edges, nodes)

    def get_nodes_within(self, t):
        nodes = nx.shortest_path(
            self.experiment.nx_graph[t],
            source=self.root.label,
            target=self.end.label,
            weight="weight",
        )
        edges = [
            Edge(
                self.experiment.get_node(nodes[i]),
                self.experiment.get_node(nodes[i + 1]),
                self.experiment,
            )
            for i in range(len(nodes) - 1)
        ]
        return (nodes, edges)

    def get_mother(self):
        candidate_mother = []
        for hyphae in self.experiment.hyphaes:
            for t in hyphae.ts:
                if self.root in hyphae.get_nodes_within(t):
                    candidate_mother.append(hyphae)
        self.mother = candidate_mother

    def update_ts(self):
        self.ts = sorted(set(self.end.ts()).intersection(set(self.root.ts())))


def get_hyphae(experiment):
    tips = [
        node
        for node in experiment.nodes
        if node.degree(node.ts()[0]) == 1 and node.pos(node.ts()[0])[0] <= 19500
    ]
    problems = []
    hyphaes = []
    #     for i in range(20):
    for i, tip in enumerate(tips):
        if i % 200 == 0:
            print(i / len(tips))
        #         tip = choice(tips)
        hyphae = Hyphae(tip)
        roots = []
        for t in tip.ts():
            #             print(t,tip)
            if tip.degree(t) == 1:
                root, edges, nodes = hyphae.get_edges(t, 50)
                roots.append(root)
        occurence_count = Counter(roots)
        if (
            len(occurence_count.values()) >= 2
            and occurence_count.most_common(2)[0][0] != roots[0]
            and occurence_count.most_common(2)[1][1]
            / occurence_count.most_common(2)[0][1]
            >= 0.75
        ):
            problems.append(tip)
        else:
            hyphae.root = occurence_count.most_common(2)[0][0]
            hyphae.ts = sorted(set(hyphae.ts).intersection(set(hyphae.root.ts())))
            hyphaes.append(hyphae)
    return (hyphaes, problems)


def reconnect_degree_2(nx_graph, pos):
    degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node) == 2]
    while len(degree_2_nodes) > 0:
        node = degree_2_nodes.pop()
        neighbours = list(nx_graph.neighbors(node))
        right_n = neighbours[0]
        left_n = neighbours[1]
        right_edge = nx_graph.get_edge_data(node, right_n)["pixel_list"]
        left_edge = nx_graph.get_edge_data(node, left_n)["pixel_list"]
        if np.any(right_edge[0] != pos[node]):
            right_edge = list(reversed(right_edge))
        if np.any(left_edge[-1] != pos[node]):
            left_edge = list(reversed(left_edge))
        pixel_list = left_edge + right_edge[1:]
        info = {"weight": len(pixel_list), "pixel_list": pixel_list}
        if right_n != left_n:
            connection_data = nx_graph.get_edge_data(right_n, left_n)
            if connection_data is None or connection_data["weight"] >= info["weight"]:
                if not connection_data is None:
                    nx_graph.remove_edge(right_n, left_n)
                nx_graph.add_edges_from([(right_n, left_n, info)])
        nx_graph.remove_node(node)
        degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node) == 2]


def clean_exp_with_hyphaes(experiment):
    ts = {}
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
    exp_clean = Experiment(experiment.plate)
    exp_clean.copy(experiment)
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes = []
    for label in labels:
        exp_clean.nodes.append(Node(label, exp_clean))
    for node in exp_clean.nodes:
        ts[node.label] = []
        for t in range(len(nx_graph_cleans)):
            if node.is_in(t):
                ts[node.label].append(t)
    print("before getting hyphaes", 3 in exp_clean.get_node(1354).ts())
    hyphaes, problems = get_hyphae(exp_clean)
    print("after getting hyphaes", 3 in exp_clean.get_node(1354).ts())
    to_remove = []
    safe_nodes = set()
    roots = set()
    tips = set()
    for hyphae in hyphaes:
        roots.add(hyphae.root.label)
        tips.add(hyphae.end.label)
        for t in hyphae.end.ts():
            if t in hyphae.root.ts():
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if (
            len(ts[node.label]) == 1
            and ts[node.label][0] != len(exp_clean.nx_graph) - 1
            and node.label not in safe_nodes
        ):
            to_remove.append(node.label)
    #     return(to_remove)
    print("removing ", len(to_remove), " nodes")
    print("removing", 1354 in to_remove)
    for node in to_remove:
        t = ts[node][0]
        pos = exp_clean.positions[t]
        nx_graph_clean = nx_graph_cleans[t]
        #         if t ==3:
        #             print('before everythin',node,node_to_fuse,1354 in nx_graph_clean)
        if nx_graph_clean.degree(node) <= 2:
            continue
        #         print('cleaning',node)
        neighbours = list(nx_graph_clean.neighbors(node))
        candidate_to_fuse = []
        weight_candidate = []
        for neighbour in neighbours:
            if nx_graph_clean.degree(neighbour) >= 3:
                candidate_to_fuse.append(neighbour)
                weight_candidate.append(
                    len(nx_graph_clean.get_edge_data(node, neighbour)["pixel_list"])
                    - 100 * (neighbour in roots)
                    - 100 * (neighbour in tips)
                )
        if len(weight_candidate) == 0:
            continue
        node_to_fuse = candidate_to_fuse[np.argmin(weight_candidate)]
        if nx_graph_clean.degree(node) == 1 and node_to_fuse not in to_remove:
            print(pos[node])
            continue
        for neighbour in neighbours:
            right_n = node_to_fuse
            left_n = neighbour
            right_edge = nx_graph_clean.get_edge_data(node, right_n)["pixel_list"]
            left_edge = nx_graph_clean.get_edge_data(node, left_n)["pixel_list"]
            if np.any(right_edge[0] != pos[node]):
                right_edge = list(reversed(right_edge))
            if np.any(left_edge[-1] != pos[node]):
                left_edge = list(reversed(left_edge))
            pixel_list = left_edge + right_edge[1:]
            info = {"weight": len(pixel_list), "pixel_list": pixel_list}
            if right_n != left_n:
                connection_data = nx_graph_clean.get_edge_data(right_n, left_n)
                if (
                    connection_data is None
                    or connection_data["weight"] >= info["weight"]
                ):
                    if not connection_data is None:
                        nx_graph_clean.remove_edge(right_n, left_n)
                    nx_graph_clean.add_edges_from([(right_n, left_n, info)])
        #         if t ==3:
        #             print('before removinb node',node,node_to_fuse,1354 in nx_graph_clean)
        nx_graph_clean.remove_node(node)
        #         if t ==3:
        #             print('after removinb node',node,node_to_fuse,1354 in nx_graph_clean)
        nx_graph_cleans[t] = nx_graph_clean
    for t, nx_graph in enumerate(nx_graph_cleans):
        pos = exp_clean.positions[t]
        reconnect_degree_2(nx_graph, pos)
    print("before pruning", 1354 in nx_graph_cleans[3])
    nx_graph_pruned = []
    for graph in nx_graph_cleans:
        S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
        len_connected = [len(nx_graph.nodes) for nx_graph in S]
        nx_graph_pruned.append(S[np.argmax(len_connected)])
    print("after pruning", 1354 in nx_graph_pruned[3])
    skeletons = []
    for nx_graph in nx_graph_pruned:
        skeletons.append(generate_skeleton(nx_graph, dim=(20800, 46000)))
    print("before changing nx_graph", 3 in exp_clean.get_node(1354).ts())
    exp_clean.nx_graph = nx_graph_pruned
    print("after changing nx_graph", 3 in exp_clean.get_node(1354).ts())
    exp_clean.skeletons = skeletons
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes = []
    for hyphae in hyphaes:
        hyphae.update_ts()
    exp_clean.hyphaes = hyphaes
    for label in labels:
        exp_clean.nodes.append(Node(label, exp_clean))
    return exp_clean

In [29]:
100 * True

100

In [15]:
3 in exp.get_node(1354).ts()

True